# Criando uma tabela no MySQL com as informações de fatura do mês 09/2023 até 02/2024

Solicitei que o banco enviasse por e-mail os arquivos CSV referente às faturas dos últimos 6 meses do meu cartão de crédito, para que eu pudesse fazer uma análise dos meus gastos recentes e construir um dashboard no PowerBI.

## Lendo os arquivos CSV

In [1]:
import pandas as pd
import mysql.connector
# Import para utilizar variáveis de ambiente do arquivo .env
from dotenv import load_dotenv
import os

load_dotenv('.env')

True

In [2]:
# Utilizando a função da biblioteca pandas, fiz a leitura do arquivo csv referente a fatura de cada mês e inseri num dataframe.
df1 = pd.read_csv('faturas/fevereiro/Fatura_2024-02-20.csv', sep=';')
df2 = pd.read_csv('faturas/janeiro/Fatura_2024-01-20.csv', sep=';')
df3 = pd.read_csv('faturas/dezembro/Fatura_2023-12-20.csv', sep=';')
df4 = pd.read_csv('faturas/novembro/Fatura_2023-11-20.csv', sep=';')
df5 = pd.read_csv('faturas/outubro/Fatura_2023-10-20.csv', sep=';')
df6 = pd.read_csv('faturas/setembro/Fatura_2023-09-20.csv', sep=';')

In [3]:
# Dessa forma, juntei todas as faturas em um único dataframe que servirá de base para a tabela no MySQL.
df_final = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)

In [4]:
df_final.shape

(185, 9)

In [5]:
df_final.head()

,Data de Compra,Nome no Cartão,Final do Cartão,Categoria,Descrição,Parcela,Valor (em US$),Cotação (em R$),Valor (em R$)
0,10/05/2023,GUSTAVO CAVALCANTE,1342,Serviços pessoais,HNA O BOTICARIO,9/10,0.0,0.0,13.18
1,01/10/2023,GUSTAVO CAVALCANTE,1342,Serviços pessoais,HNA *O BOTICARIO,5/7,0.0,0.0,20.52
2,15/12/2023,GUSTAVO CAVALCANTE,1342,Departamento / Desconto,ZATTINI,2/4,0.0,0.0,64.97
3,15/12/2023,GUSTAVO CAVALCANTE,1342,Departamento / Desconto,ZATTINI,2/2,0.0,0.0,83.24
4,15/12/2023,GUSTAVO CAVALCANTE,1342,Departamento / Desconto,MLP *ZATTINI,2/10,0.0,0.0,87.43


In [6]:
df_final.tail()

,Data de Compra,Nome no Cartão,Final do Cartão,Categoria,Descrição,Parcela,Valor (em US$),Cotação (em R$),Valor (em R$)
180,02/09/2023,GUSTAVO CAVALCANTE,8114,Vestuário / Roupas,PAG DAHORASKTSHOP,1/4,0.0,0.0,25.00
181,02/09/2023,GUSTAVO CAVALCANTE,8114,Departamento / Desconto,PAG ELLENBRIONELIGERO,Única,0.0,0.0,95.00
182,07/09/2023,GUSTAVO CAVALCANTE,8114,Departamento / Desconto,LOJAS MEL,Única,0.0,0.0,15.98
183,12/09/2023,GUSTAVO CAVALCANTE,8114,Transporte,AUTOPASS S.A*DESCRIPTI,Única,0.0,0.0,6.80
184,12/09/2023,GUSTAVO CAVALCANTE,8114,Supermercados / Mercearia / Padarias / Lojas d...,COMERCIAL MARUKAI LTDA,Única,0.0,0.0,20.20


## Criando a tabela no banco

In [7]:
# Depois de importar a biblioteca de conexão com o MySQL, usamos a função para conectar com o banco, passando os parâmentos do
# servidor, usuário, senha e nome do banco de dados.
# 'projeto-financas' é o nome do banco de dados que eu criei previamente no MySQL.
# 'servidor', 'usuario' e 'senha' são variáveis de ambiente que eu criei dentro do arquivo .env para não expor os dados de
# conexão com o banco de dados.
connection = mysql.connector.connect(
    host=os.getenv('servidor'),
    user=os.getenv('usuario'),
    password=os.getenv('senha'),
    database='projeto-financas'
)
cursor = connection.cursor()

In [8]:
# Dentro da variável inserimos o comando de texto para a criação da tabela dentro do banco de dados, baseado em cada coluna do
# dataframe e seu tipo de dado. Depois, o cursor executa a criação da tabela.
create_table_query = """
CREATE TABLE IF NOT EXISTS gastos_cartao (
    id INT AUTO_INCREMENT PRIMARY KEY,
    data_compra VARCHAR(50),
    nome VARCHAR(100),
    final_cartao INT,
    categoria VARCHAR(100),
    descricao VARCHAR(100),
    parcela VARCHAR(100),
    valor_dolar FLOAT,
    cotacao FLOAT,
    valor_reais FLOAT
)
"""
cursor.execute(create_table_query)

In [9]:
# Com essa estrutura de repetição, cada linha do dataframe é lida de forma que inserimos no campo específico da tabela 
# gastos_cartao o valor correspondente 
for indice, linha in df_final.iterrows():
    insert_query = f"""INSERT INTO gastos_cartao (data_compra, nome, final_cartao, categoria, descricao, parcela, 
    valor_dolar, cotacao, valor_reais) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    valores = tuple(linha)
    cursor.execute(insert_query, valores)

In [10]:
# O commit finalmente executa os comandos realizando a gravação no banco de dados e após isso finalizamos a conexão.
connection.commit()
connection.close()